In [1]:
!pip install gensim

In [2]:
!pip install -U numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 55.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/KDD/MovieReviews' # Replace with the actual path to your folder

# List files in the folder
try:
    files = os.listdir(folder_path)
    print("Files found the folder:")

except FileNotFoundError:
    print(f"Error: Folder not found at {folder_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files found the folder:


In [5]:
import os

import pandas as pd


file_path = '/content/drive/MyDrive/KDD/MovieReviews2'
documents = []


try:
    files = os.listdir(folder_path)
    print(f"Files found in the folder: {len(files)}")

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='latin-1') as f:
            content = f.read()
            documents.append(content)

    print(f"Successfully loaded {len(documents)} documents.")

except FileNotFoundError:
    print(f"Error: Folder not found at {folder_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Files found in the folder: 180
Successfully loaded 180 documents.


In [6]:
#Load and preprocess the corpus data from "/content/drive/MyDrive/KDD/MovieReviews 2.csv" into the Python session, making the data ready for topic modeling.
#Load the data from the specified file path into a pandas DataFrame.

df = pd.DataFrame(documents, columns=['review'])
print("DataFrame Head:")
display(df.head())
print("\nDataFrame Info:")
df.info()

DataFrame Head:


,review
0,Date Released: 5/7/93 Running Length: 2:00 Rat...
1,"Starring: John D. LeMay, Kari Keegan, Erin Gra..."
2,"Dead man seen dancing in a Congo line, picking..."
3,Date Released: 6/18/93 Running Length: 1:53 Ra...
4,"Two twins, separated at birth, grow into adult..."



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  180 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB


In [7]:
#Preprocess the text data by removing punctuation, numbers, and converting text to lowercase.



import re
import string

def clean_text(text):
    """Converts text to lowercase, removes punctuation and numbers."""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text

df['cleaned_review'] = df['review'].apply(clean_text)

print("DataFrame with cleaned reviews:")
display(df.head())

DataFrame with cleaned reviews:


,review,cleaned_review
0,Date Released: 5/7/93 Running Length: 2:00 Rat...,date released running length rated pg violen...
1,"Starring: John D. LeMay, Kari Keegan, Erin Gra...",starring john d lemay kari keegan erin gray st...
2,"Dead man seen dancing in a Congo line, picking...",dead man seen dancing in a congo line picking ...
3,Date Released: 6/18/93 Running Length: 1:53 Ra...,date released running length rated pg mature...
4,"Two twins, separated at birth, grow into adult...",two twins separated at birth grow into adultho...


In [8]:
#Split the text into individual words or tokens.

import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt' tokenizer data
nltk.download('punkt_tab')

df['tokens'] = df['cleaned_review'].apply(word_tokenize)

print("DataFrame with tokens:")
display(df.head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


DataFrame with tokens:


,review,cleaned_review,tokens
0,Date Released: 5/7/93 Running Length: 2:00 Rat...,date released running length rated pg violen...,"[date, released, running, length, rated, pg, v..."
1,"Starring: John D. LeMay, Kari Keegan, Erin Gra...",starring john d lemay kari keegan erin gray st...,"[starring, john, d, lemay, kari, keegan, erin,..."
2,"Dead man seen dancing in a Congo line, picking...",dead man seen dancing in a congo line picking ...,"[dead, man, seen, dancing, in, a, congo, line,..."
3,Date Released: 6/18/93 Running Length: 1:53 Ra...,date released running length rated pg mature...,"[date, released, running, length, rated, pg, m..."
4,"Two twins, separated at birth, grow into adult...",two twins separated at birth grow into adultho...,"[two, twins, separated, at, birth, grow, into,..."


In [9]:
#Eliminate common English stop words from the tokenized text.

import nltk
from nltk.corpus import stopwords

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    """Removes English stop words from a list of tokens."""
    return [word for word in tokens if word not in stop_words]

df['tokens_without_stopwords'] = df['tokens'].apply(remove_stopwords)

print("DataFrame with tokens after removing stop words:")
display(df.head())

DataFrame with tokens after removing stop words:


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,review,cleaned_review,tokens,tokens_without_stopwords
0,Date Released: 5/7/93 Running Length: 2:00 Rat...,date released running length rated pg violen...,"[date, released, running, length, rated, pg, v...","[date, released, running, length, rated, pg, v..."
1,"Starring: John D. LeMay, Kari Keegan, Erin Gra...",starring john d lemay kari keegan erin gray st...,"[starring, john, d, lemay, kari, keegan, erin,...","[starring, john, lemay, kari, keegan, erin, gr..."
2,"Dead man seen dancing in a Congo line, picking...",dead man seen dancing in a congo line picking ...,"[dead, man, seen, dancing, in, a, congo, line,...","[dead, man, seen, dancing, congo, line, pickin..."
3,Date Released: 6/18/93 Running Length: 1:53 Ra...,date released running length rated pg mature...,"[date, released, running, length, rated, pg, m...","[date, released, running, length, rated, pg, m..."
4,"Two twins, separated at birth, grow into adult...",two twins separated at birth grow into adultho...,"[two, twins, separated, at, birth, grow, into,...","[two, twins, separated, birth, grow, adulthood..."


In [10]:
#Reduce words to their base or dictionary form.

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

try:
    wordnet.ensure_loaded()
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4') # Often needed with wordnet

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatizes a list of tokens."""
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

df['lemmatized_tokens'] = df['tokens_without_stopwords'].apply(lemmatize_tokens)

print("DataFrame with lemmatized tokens:")
display(df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


DataFrame with lemmatized tokens:


,review,cleaned_review,tokens,tokens_without_stopwords,lemmatized_tokens
0,Date Released: 5/7/93 Running Length: 2:00 Rat...,date released running length rated pg violen...,"[date, released, running, length, rated, pg, v...","[date, released, running, length, rated, pg, v...","[date, released, running, length, rated, pg, v..."
1,"Starring: John D. LeMay, Kari Keegan, Erin Gra...",starring john d lemay kari keegan erin gray st...,"[starring, john, d, lemay, kari, keegan, erin,...","[starring, john, lemay, kari, keegan, erin, gr...","[starring, john, lemay, kari, keegan, erin, gr..."
2,"Dead man seen dancing in a Congo line, picking...",dead man seen dancing in a congo line picking ...,"[dead, man, seen, dancing, in, a, congo, line,...","[dead, man, seen, dancing, congo, line, pickin...","[dead, man, seen, dancing, congo, line, pickin..."
3,Date Released: 6/18/93 Running Length: 1:53 Ra...,date released running length rated pg mature...,"[date, released, running, length, rated, pg, m...","[date, released, running, length, rated, pg, m...","[date, released, running, length, rated, pg, m..."
4,"Two twins, separated at birth, grow into adult...",two twins separated at birth grow into adultho...,"[two, twins, separated, at, birth, grow, into,...","[two, twins, separated, birth, grow, adulthood...","[two, twin, separated, birth, grow, adulthood,..."


In [11]:
#Create a dictionary and a corpus (bag-of-words representation) from the preprocessed text, which are required formats for many topic modeling algorithms.

from gensim.corpora import Dictionary

# Create a Gensim Dictionary from the lemmatized tokens
dictionary = Dictionary(df['lemmatized_tokens'])

# Create a Gensim Corpus (bag-of-words representation)
corpus = [dictionary.doc2bow(tokens) for tokens in df['lemmatized_tokens']]

# Print information about the dictionary and corpus
print("Gensim Dictionary and Corpus created.")
print(f"Number of unique tokens in the dictionary: {len(dictionary)}")
print(f"Number of documents in the corpus: {len(corpus)}")
print("\nFirst 5 documents in the corpus (bag-of-words representation):")
for i, doc in enumerate(corpus[:5]):
    print(f"Document {i+1}: {doc}")


Gensim Dictionary and Corpus created.
Number of unique tokens in the dictionary: 14234
Number of documents in the corpus: 180

First 5 documents in the corpus (bag-of-words representation):
Document 1: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 12), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 5), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 3), (81, 1), (82, 4), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 



### Data Analysis Key Findings

*   The initial attempt to load the data from the specified CSV path failed because the file was not located there. The data was successfully loaded into a DataFrame from an existing Python variable named `documents`, which contained 180 movie reviews.
*   The text data was successfully cleaned by converting it to lowercase and removing punctuation and numbers, resulting in a new column `cleaned_review`.
*   The cleaned text was successfully tokenized into individual words and stored in the `tokens` column after downloading the necessary NLTK 'punkt' and 'punkt_tab' data.
*   Common English stop words were removed from the tokens, creating the `tokens_without_stopwords` column.
*   The tokens were lemmatized to their base form using the WordNet Lemmatizer after downloading the 'wordnet' and 'omw-1.4' NLTK data, and the results were stored in the `lemmatized_tokens` column.
*   A Gensim Dictionary was created from the lemmatized tokens, containing 14234 unique tokens.
*   A Gensim Corpus (bag-of-words representation) was created for the 180 documents, which is a format suitable for topic modeling algorithms.

### Insights or Next Steps

*   The preprocessed data, including the Gensim Dictionary and Corpus, is now ready to be used as input for training a topic modeling algorithm such as Latent Dirichlet Allocation (LDA).
*   Further analysis could involve exploring the most frequent terms in the dictionary or visualizing the corpus structure.


In [12]:
#Correctly apply the LDA topic modeling function and get the correct topic modeling result (topic IDs and their top 10 words).

from gensim.models import LdaModel

# Set the number of topics
num_topics = 5  # we can adjust this number as needed

# Train the LDA model
# 'corpus' and 'dictionary' were created in previous steps
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)


# Print the topics and their top words
print(f"LDA Model with {num_topics} topics:")
for topic_id, topic_words in lda_model.print_topics(num_words=10):
    print(f"Topic {topic_id}: {topic_words}")

LDA Model with 5 topics:
Topic 0: 0.019*"film" + 0.008*"one" + 0.006*"story" + 0.005*"get" + 0.005*"life" + 0.004*"see" + 0.004*"even" + 0.004*"movie" + 0.004*"director" + 0.004*"dennis"
Topic 1: 0.015*"film" + 0.006*"one" + 0.004*"like" + 0.004*"movie" + 0.004*"story" + 0.004*"life" + 0.004*"time" + 0.003*"make" + 0.003*"get" + 0.003*"see"
Topic 2: 0.011*"film" + 0.011*"movie" + 0.006*"one" + 0.004*"character" + 0.004*"picture" + 0.004*"there" + 0.004*"like" + 0.004*"scene" + 0.003*"time" + 0.003*"get"
Topic 3: 0.012*"film" + 0.006*"one" + 0.005*"story" + 0.004*"life" + 0.003*"movie" + 0.003*"make" + 0.003*"even" + 0.003*"character" + 0.003*"director" + 0.003*"good"
Topic 4: 0.013*"film" + 0.007*"one" + 0.005*"even" + 0.004*"get" + 0.004*"movie" + 0.004*"life" + 0.003*"story" + 0.003*"see" + 0.003*"like" + 0.003*"make"


In [ ]:
# Print the topics
print(topic_words)

0.014*"film" + 0.005*"one" + 0.005*"movie" + 0.005*"story" + 0.004*"get" + 0.004*"life" + 0.004*"make" + 0.003*"time" + 0.003*"father" + 0.003*"like"


In [13]:
#Correctly apply the print topics function for all the documents in the corpus and get the correct result (topic coverage scores for each document in the corpus).

# Get topic coverage scores for each document
print("Topic coverage scores for each document:")
for i, doc_topics in enumerate(lda_model[corpus]):
    print(f"Document {i+1}: {doc_topics}")

Topic coverage scores for each document:
Document 1: [(2, 0.99756604)]
Document 2: [(2, 0.998001)]
Document 3: [(2, 0.99771816)]
Document 4: [(2, 0.99767995)]
Document 5: [(2, 0.9982998)]
Document 6: [(0, 0.5974537), (2, 0.4011697)]
Document 7: [(2, 0.99863267)]
Document 8: [(2, 0.9977058)]
Document 9: [(2, 0.9976585)]
Document 10: [(2, 0.99826866)]
Document 11: [(1, 0.9973972)]
Document 12: [(2, 0.9980557)]
Document 13: [(2, 0.99763703)]
Document 14: [(2, 0.5343114), (4, 0.46382618)]
Document 15: [(2, 0.9982031)]
Document 16: [(1, 0.75343543), (2, 0.2454845)]
Document 17: [(1, 0.8585683), (2, 0.14010024)]
Document 18: [(2, 0.9979172)]
Document 19: [(2, 0.23490636), (4, 0.7634333)]
Document 20: [(2, 0.9980867)]
Document 21: [(2, 0.99794984)]
Document 22: [(1, 0.83474475), (2, 0.16354135)]
Document 23: [(2, 0.21291897), (3, 0.7852544)]
Document 24: [(2, 0.99785715)]
Document 25: [(0, 0.74836177), (2, 0.25029638)]
Document 26: [(2, 0.9989968)]
Document 27: [(2, 0.083065495), (3, 0.915482